In [7]:
import json
import pandas as pd
import os
import numpy as np

In [13]:
df = pd.read_csv("aws_dea_questions_final.csv")

# Biến đổi các kí tự "/n/n" thành "<br><br>"
df['question_text'] = df['question_text'].str.replace('''/n/n''', '<br><br>')
df['question_text'] = df['question_text'].str.replace('''\n\n''', '<br><br>')

df = df [['question_text', 'correct_answer', 'A','B', 'C', 'D','E']]

type1 = df[df['correct_answer'].str.len() == 1]
type2 = df[df['correct_answer'].str.len() > 1]

type1_split = np.array_split(type1, 23)
type2_split = np.array_split(type2, 23)

file_order = [0] * len(df)
for i in range(23):
    idx_type1 = type1_split[i].index.tolist()
    idx_type2 = type2_split[i].index.tolist()
    for idx in idx_type1 + idx_type2:
        file_order[idx] = i + 1

df['exam'] = file_order

/home/phongnl/exam_DEA/Quiz_app/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
df

,question_text,correct_answer,A,B,C,D,E,exam
0,A data engineer is configuring an AWS Glue job...,D,Update the AWS Glue security group to allow in...,Configure an S3 bucket policy to explicitly gr...,Review the AWS Glue job code to ensure that th...,Verify that the VPC's route table includes inb...,NaN,1
1,A retail company has a customer data hub in an...,B,Create a separate table for each country's cus...,Register the S3 bucket as a data lake location...,Move the data to AWS Regions that are close to...,Load the data into Amazon Redshift. Create a v...,NaN,1
2,A media company wants to improve a system that...,A,Use API calls to access and integrate third-pa...,Use API calls to access and integrate third-pa...,Use Amazon Kinesis Data Streams to access and ...,Use Amazon Kinesis Data Streams to access and ...,NaN,1
3,A financial company wants to implement a data ...,BE,Use Amazon Aurora for data storage. Use an Ama...,Use Amazon S3 for data storage. Use Amazon Ath...,Use AWS Glue DataBrew for centralized data gov...,Use Amazon RDS for data storage. Use Amazon EM...,Use AWS Lake Formation for centralized data go...,1
4,A data engineer maintains custom Python script...,B,Store a pointer to the custom Python scripts i...,Package the custom Python scripts into Lambda ...,Store a pointer to the custom Python scripts i...,Assign the same alias to each Lambda function....,NaN,1
...,...,...,...,...,...,...,...,...
202,A company has a data warehouse that contains a...,B,Select * from Sales where city_name ~ ‘$(San|E...,Select * from Sales where city_name ~ ‘^(San|E...,Select * from Sales where city_name ~’$(San&El...,Select * from Sales where city_name ~ ‘^(San&E...,NaN,23
203,A company needs to send customer call data fro...,D,Use Amazon CloudWatch to monitor the DMS task....,Verify that logical replication of the source ...,Enable Amazon CloudWatch Logs for the DMS endp...,Use Amazon CloudWatch to monitor the DMS task....,NaN,23
204,"A lab uses IoT sensors to monitor humidity, te...",C,Use Amazon Kinesis Data Streams and Amazon Kin...,Use Amazon Kinesis Data Streams to deliver the...,Use Amazon Kinesis Data Streams and call the K...,Use Amazon Managed Service for Apache Flink (p...,NaN,23
205,A company wants to use machine learning (ML) t...,AD,"For daily incoming data, use AWS Glue crawlers...","For daily incoming data, use Amazon Athena to ...","For daily incoming data, use Amazon Redshift t...","For daily and archived data, use Amazon EMR to...","For archived data, use Amazon SageMaker to per...",23


# Convert to json

In [15]:
for exam_num in range(1, 24):
    df_subset = df[df['exam'] == exam_num]
    file_name = f'questions_{exam_num}.json'
    if df_subset.empty:
        continue
    questions_list = []
    for _, row in df_subset.iterrows():
        # Lấy tất cả các cột chứa đáp án (A, B, C, D, E,...)
        options_cols = [col for col in row.index if col not in ['question_text', 'correct_answer', 'exam']]
        question_dict = {
            "question": row['question_text'],
            "options": [row[col] for col in options_cols if pd.notnull(row[col])],
            "answer": row['correct_answer'] if len(row['correct_answer']) == 1 else list(row['correct_answer']),
            "type": "single" if len(row['correct_answer']) == 1 else "multiple"
        }
        questions_list.append(question_dict)
    
    # Lưu vào file JSON
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(questions_list, f, ensure_ascii=False, indent=2)
    # break